# Часть 1

Поставить python и все библиотеки

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score

import numpy as np

In [2]:
iris = load_iris()

X = iris.data[:, :2]
y = iris.target

print round(np.mean(cross_val_score(RandomForestClassifier(random_state=42), X, y, cv=3)), 4)

0.7132


#### Вопрос 1. Что выведет этот код?

# Часть 2


In [104]:
iris = load_iris()

X = iris.data[:, :2]
y = iris.target

#### Вопрос 2. Каков минимальный радиус круга, содержащего все точки из $X$? Ответ округлите до 4 знака после запятой.

In [105]:
from scipy.optimize import minimize

Первый способ

In [106]:
minimize(lambda point: ((point - X) ** 2)
         .sum(axis=1).max(), np.mean(X, axis=0))

      fun: 3.468291571416955
 hess_inv: array([[ 0.00852046, -0.00986477],
       [-0.00986477,  0.0114212 ]])
      jac: array([ 2.39971387,  0.83110574])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 456
      nit: 3
     njev: 111
   status: 2
  success: False
        x: array([ 6.14927676,  3.16497268])

In [107]:
min_value = 1000000.
for x0 in np.linspace(6., 7., 10):
    for x1 in np.linspace(3., 4., 10):
        min_value = min(min_value, minimize(lambda point: ((point - X) ** 2).sum(axis=1).max(), np.array([x0, x1])).fun)

In [108]:
print round(np.sqrt(min_value), 4)

1.8614


Второй способ

In [109]:
minimize(lambda point: ((point - X) ** 2)
         .sum(axis=1).max(), np.mean(X, axis=0), method='Nelder-Mead')

 final_simplex: (array([[ 6.15520434,  3.15154545],
       [ 6.15517458,  3.15160564],
       [ 6.15519425,  3.15154353]]), array([ 3.46483107,  3.46483507,  3.46484291]))
           fun: 3.4648310736449393
       message: 'Optimization terminated successfully.'
          nfev: 68
           nit: 36
        status: 0
       success: True
             x: array([ 6.15520434,  3.15154545])

In [110]:
min_value = minimize(lambda point: ((point - X) ** 2).sum(axis=1).max(), np.mean(X, axis=0), method='Nelder-Mead').fun
round(np.sqrt(min_value), 4)

1.8614

https://ru.wikipedia.org/wiki/Пауль_(осьминог)

Этот осьминог предсказывал результаты футбольных матчей на  ЧЕ-2008 и ЧМ-2010. Нужно проанализировать его предсказания, считая, что футбольный матч обязательно заканчивается победой одной из команд, а осьминог возможно выбирал равновероятно наугад.

#### Вопрос 3. Какова вероятность получить именно те предсказания, что дал осьминог, выбирая наугад? Ответ округлите до 5 знака после запятой.

In [7]:
print '{:0.5f}'.format(1. / 2 ** 14)

0.00006


#### Вопрос 4. Какова вероятность ошибиться не меньшее число раз, чем осьминог Пауль, выбирая наугад? Ответ округлите до 5 знака после запятой.

In [8]:
print '{:0.5f}'.format((1. + 14. + 14 * 13 / 2) / 2 ** 14)

0.00647


# Часть 3


#### Вопрос 5. У вас есть набор зависимости $y$ от одной переменной $x$, причём вы знаете, что эта зависимость линейная ($y=ax + b$). Вам нужно восстановить эту зависимость, решив оптимизационную задачу $\min_{a, b} \sum_{i=1}^n (ax_i + b - y_i)^2$.

#### Проблема в том, что выборка очень большая, поэтому её нельзя полностью загрузить в память. Ответом на вопрос является строка в питоновском формате '{}_{}'.format(round(a, 5), round(b, 5)), например, '0.00001_0.00002'.

In [9]:
def generator():
    rand_gen = np.random.RandomState(42)
    for i in xrange(100000):
        arr_of_x = 10 * rand_gen.uniform(size=10000)
        arr_of_noise = 40 * rand_gen.normal(size=10000)
        for x, noise in zip(arr_of_x, arr_of_noise):
            yield x, 3.14 * x + 8.3 + noise

In [10]:
def solver(seq):
    count = 0
    sum_x = 0.
    sum_y = 0.
    sum_sq_x = 0.
    sum_xy = 0.
    for x, y in seq:
        count += 1
        sum_x += x
        sum_y += y
        sum_sq_x += x ** 2
        sum_xy += x * y
        
    a = (count * sum_xy - sum_x * sum_y) / (count * sum_sq_x - sum_x ** 2)
    b = (sum_y - a * sum_x) / count
    return a, b
        

In [11]:
%%time
a, b = solver(generator())
print '{}_{}'.format(round(a, 5), round(b, 5))

3.13957_8.30331
CPU times: user 22min 18s, sys: 15.5 s, total: 22min 34s
Wall time: 22min 36s


#### Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

#### В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через "," затем идёт ";" после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, "1,2,3,4;" или "1,2,3,4;5,6". Гарантируется, что среди id купленных товаров все различные.

#### Вопросы 6-7. Будем считать, что цена товара равна его id % 100 + 1. Найдите разницу денежного recall для сортировки по вероятности покупки (методы уже реализованы) без умножения на цену и с умножением на цену для train и для test. Ответ округлите до 5 знака после запятой.

#### У вас есть код для подсчёта обычного recall, вам нужно переписать его, взвесив объекты их ценой. Знаменатель будет равен сумме цен купленных товаров.



In [12]:
from collections import Counter
from collections import OrderedDict
import numpy as np
import pandas
import heapq

In [13]:
visit_popularity = Counter()
purchase_popularity = Counter()

with open('train.txt', 'r') as f:
    for line in f.xreadlines():
        visits, purchases = line.strip().split(';')
        for visit in visits.split(','):
            visit_popularity[visit] += 1
        if purchases != '':
            for purchase in purchases.split(','):
                purchase_popularity[purchase] += 1

                
def get_item_price(item_id):
    return int(item_id) % 100 + 1

In [14]:
def recommend_by_purchase(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: purchase_popularity.get(x, 0))

def recommend_by_purchase_price_aware(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: get_item_price(x) * purchase_popularity.get(x, 0))

In [15]:
def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        sessions_count = 0
        for line in f.xreadlines():
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = set(purchases.split(','))
                purchases_prices = np.array([get_item_price(x) for x in purchases])

                rec = recommend(visits, max_count)
                rec_hits = np.array([get_item_price(x) * (x in purchases) for x in rec])
                hits = np.zeros(max_count)
                hits[:len(rec_hits)] = rec_hits
                
                sessions_count += 1
                avg_recall +=  np.cumsum(hits) / np.sum(purchases_prices)

    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': avg_recall / sessions_count,
    }).set_index('k')

In [16]:
print round((
    calculate_metrics(recommend_by_purchase_price_aware, 3, 'train.txt') 
    - calculate_metrics(recommend_by_purchase, 3, 'train.txt')
)['avg_recall@k'][3], 5)

0.00973


In [17]:
print round((
    calculate_metrics(recommend_by_purchase_price_aware, 3, 'test.txt') 
    - calculate_metrics(recommend_by_purchase, 3, 'test.txt')
)['avg_recall@k'][3], 5)

-0.00273
